In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("adityakadiwal/water-potability")

df=pd.read_csv(f'{path}/water_potability.csv')
df.head()

# Exploratory Data Analysis

In [ ]:
df.shape

In [ ]:
df.isnull().sum()

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df['Sulfate'].mean()

In [ ]:
df.fillna(df.mean(), inplace=True)
df.head()

In [ ]:
df.isnull().sum()

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df.Potability.value_counts()

In [ ]:
df.Potability.value_counts().plot(kind="bar", color=["brown", "salmon"])
plt.show()

In [ ]:
sns.distplot(df['ph'])

In [ ]:
df.hist(figsize=(14,14))
plt.show()

In [ ]:
sns.pairplot(df,hue='Potability')

In [ ]:
sns.scatterplot(x=df['Hardness'],y=df['Solids'])

In [ ]:
sns.scatterplot(x=df['ph'],y=df['Potability'])

In [ ]:
# create a correlation heatmap
sns.heatmap(df.corr(),annot=True, cmap='terrain', linewidths=0.1)
fig=plt.gcf()
fig.set_size_inches(8,6)
plt.show()

In [ ]:
df.boxplot(figsize=(14,7))

In [ ]:
df['Solids'].describe()

# Partitioning

In [ ]:
X = df.drop('Potability',axis=1)

In [ ]:
Y= df['Potability']

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size= 0.2, random_state=101,shuffle=True)

In [ ]:
Y_train.value_counts()

In [ ]:
Y_test.value_counts()

# Normalization

In [ ]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()

In [ ]:
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

# Model Building

# DT

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score,confusion_matrix,precision_score
from sklearn.model_selection import GridSearchCV, StratifiedKFold
# CV hyperparameter tuning to boost macro precision and accuracy
param_grid = {
    'criterion': ['gini', 'entropy'],
    'splitter': ['best', 'random'],
    'min_samples_split': [2, 4, 6, 8, 10],
    'min_samples_leaf': [1, 2, 3, 5],
    'max_depth': [None, 5, 10, 20],
    'class_weight': [None, 'balanced'],
}
scoring = {'f1_macro': 'f1_macro', 'precision_macro': 'precision_macro', 'accuracy': 'accuracy'}
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=101)
gs = GridSearchCV(
    estimator=DecisionTreeClassifier(random_state=101),
    param_grid=param_grid,
    scoring=scoring,
    refit='f1_macro',
    cv=cv,
    n_jobs=-1,
)
gs.fit(X_train, Y_train)
print('Best params:', gs.best_params_)
print('Best CV f1_macro:', round(gs.best_score_, 4))
dt = gs.best_estimator_
dt

In [ ]:
prediction=dt.predict(X_test)
accuracy_dt=accuracy_score(Y_test,prediction)*100
accuracy_dt

In [ ]:
print("Accuracy on training set: {:.3f}".format(dt.score(X_train, Y_train)))
print("Accuracy on test set: {:.3f}".format(dt.score(X_test, Y_test)))

In [ ]:
accuracy_score(prediction,Y_test)

In [ ]:
print("Feature importances:\n{}".format(dt.feature_importances_))

In [ ]:
confusion_matrix(prediction,Y_test)

# Prediction on only one set of data

In [ ]:
X_DT=dt.predict([[5.735724, 158.318741,25363.016594,7.728601,377.543291,568.304671,13.626624,75.952337,4.732954]])

In [ ]:
X_DT

# KNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
from sklearn.model_selection import GridSearchCV, StratifiedKFold
# CV hyperparameter tuning to boost macro precision and accuracy for KNN
param_grid = {
    'n_neighbors': list(range(1, 51)),
    'weights': ['uniform', 'distance'],
    'metric': ['euclidean', 'manhattan', 'minkowski'],
    'p': [1, 2, 3],
}
scoring = {'f1_macro': 'f1_macro', 'precision_macro': 'precision_macro', 'accuracy': 'accuracy'}
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=101)
gs_knn = GridSearchCV(
    estimator=KNeighborsClassifier(),
    param_grid=param_grid,
    scoring=scoring,
    refit='f1_macro',
    cv=cv,
    n_jobs=-1,
)
gs_knn.fit(X_train, Y_train)
print('Best params (KNN):', gs_knn.best_params_)
print('Best CV f1_macro (KNN):', round(gs_knn.best_score_, 4))
knn = gs_knn.best_estimator_
knn

In [ ]:
prediction_knn=knn.predict(X_test)
accuracy_knn=accuracy_score(Y_test,prediction_knn)*100
print('accuracy_score score     : ',accuracy_score(Y_test,prediction_knn)*100,'%')

In [ ]:
confusion_matrix(prediction,Y_test)

# Hyperparameter Tuning / Model Optimization

# DT HPT

In [ ]:
dt.get_params().keys()

In [ ]:
#example of grid searching key hyperparametres for logistic regression
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import GridSearchCV

# define models and parameters
model = DecisionTreeClassifier()
criterion = ["gini", "entropy"]
splitter = ["best", "random"]
min_samples_split = [2,4,6,8,10]

# define grid search
grid = dict(splitter=splitter, criterion=criterion, min_samples_split=min_samples_split)
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
grid_search_dt = GridSearchCV(estimator=model, param_grid=grid, n_jobs=-1, cv=cv,
                           scoring='accuracy',error_score=0)
grid_search_dt.fit(X_train, Y_train)

# summarize results
print(f"Best: {grid_search_dt.best_score_:.3f} using {grid_search_dt.best_params_}")
means = grid_search_dt.cv_results_['mean_test_score']
stds = grid_search_dt.cv_results_['std_test_score']
params = grid_search_dt.cv_results_['params']

for mean, stdev, param in zip(means, stds, params):
    print(f"{mean:.3f} ({stdev:.3f}) with: {param}")

print("Training Score:",grid_search_dt.score(X_train, Y_train)*100)
print("Testing Score:", grid_search_dt.score(X_test, Y_test)*100)

In [ ]:
from sklearn.metrics import  make_scorer, classification_report
from sklearn.model_selection import cross_val_score

def classification_report_with_accuracy_score(Y_test, y_pred2):
    print (classification_report(Y_test, y_pred2)) # print classification report
    return accuracy_score(Y_test, y_pred2) # return accuracy score


nested_score = cross_val_score(grid_search_dt, X=X_train, y=Y_train, cv=cv,
               scoring=make_scorer(classification_report_with_accuracy_score))
print (nested_score)

In [ ]:
dt_y_predicted = grid_search_dt.predict(X_test)
dt_y_predicted

In [ ]:
grid_search_dt.best_params_

In [ ]:
dt_grid_score=accuracy_score(Y_test, dt_y_predicted)
dt_grid_score

In [ ]:
confusion_matrix(Y_test, dt_y_predicted)

# KNN HPT

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import GridSearchCV

# define models and parameters
model = KNeighborsClassifier()
n_neighbors = range(1, 31)
weights = ['uniform', 'distance']
metric = ['euclidean', 'manhattan', 'minkowski']

# define grid search
grid = dict(n_neighbors=n_neighbors,weights=weights,metric=metric)
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=1, random_state=1)
grid_search_knn = GridSearchCV(estimator=model, param_grid=grid, n_jobs=-1, cv=cv,
                           scoring='accuracy',error_score=0, iid=True)
grid_search_knn.fit(X_train, Y_train)

# summarize results
print(f"Best: {grid_search_knn.best_score_:.3f} using {grid_search_knn.best_params_}")
means = grid_search_knn.cv_results_['mean_test_score']
stds = grid_search_knn.cv_results_['std_test_score']
params = grid_search_knn.cv_results_['params']

for mean, stdev, param in zip(means, stds, params):
    print(f"{mean:.3f} ({stdev:.3f}) with: {param}")

In [ ]:
from sklearn.metrics import  make_scorer
from sklearn.model_selection import cross_val_score

def classification_report_with_accuracy_score(Y_test, y_pred2):
    print (classification_report(Y_test, y_pred2)) # print classification report
    return accuracy_score(Y_test, y_pred2) # return accuracy score


nested_score = cross_val_score(grid_search_knn, X=X_train, y=Y_train, cv=cv,
               scoring=make_scorer(classification_report_with_accuracy_score))
print (nested_score)

In [ ]:
knn_y_predicted = grid_search_knn.predict(X_test)

In [ ]:
knn_y_predicted

In [ ]:
knn_grid_score=accuracy_score(Y_test, knn_y_predicted)

In [ ]:
knn_grid_score

In [ ]:
grid_search_knn.best_params_

In [ ]:
confusion_matrix(Y_test, knn_y_predicted)

# Prediction on only one set of data

In [ ]:
X_KNN=knn.predict([[5.735724, 158.318741,25363.016594,7.728601,377.543291,568.304671,13.626624,75.952337,4.732954]])

In [ ]:
X_KNN